In [1]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载音频处理库

In [3]:
# 其他库

import os
import librosa
import librosa.display
import glob 

## 特征提取以及数据集的建立

In [4]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}
print(label_dict)
print(label_dict_inv)

{'aloe': 0, 'burger': 1, 'cabbage': 2, 'candied_fruits': 3, 'carrots': 4, 'chips': 5, 'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream': 11, 'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon': 17, 'soup': 18, 'wings': 19}
{0: 'aloe', 1: 'burger', 2: 'cabbage', 3: 'candied_fruits', 4: 'carrots', 5: 'chips', 6: 'chocolate', 7: 'drinks', 8: 'fries', 9: 'grapes', 10: 'gummies', 11: 'ice-cream', 12: 'jelly', 13: 'noodles', 14: 'pickles', 15: 'pizza', 16: 'ribs', 17: 'salmon', 18: 'soup', 19: 'wings'}


In [18]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
            # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            # print(fn)
            label_name = fn.split('/')[-2]
            # print(label_name)
            
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
    
    return [feature, label]



In [19]:
# 自己更改目录  
parent_dir = '../data/train_sample/'
save_dir = "../data"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label

temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:05<00:00,  6.36it/s]


In [109]:
temp = np.array(temp)
data = temp.transpose()
# 转置
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1],dtype= int)
print('X的特征尺寸是',X.shape)
print('Y的特征尺寸是',Y.shape)
from tensorflow.keras.utils import to_categorical 

# Y = to_categorical(Y)

X的特征尺寸是 (1000, 128)
Y的特征尺寸是 (1000,)


In [110]:
# 最终数据
print(X.shape)
# X = X.reshape(-1, 1, 16, 8, 1)
X = X.reshape(-1, 8, 1, 16, 1)
Y = Y.reshape(-1,1)
print(X.shape)
print(Y.shape)
print(Y.dtype)



(1000, 128)
(1000, 8, 1, 16, 1)
(1000, 1)
int64


In [95]:
import torch
import torchvision
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# from torchvision import transforms
import matplotlib.pyplot as plt
import sys, os
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [111]:
b = torch.from_numpy(X)
#c = Y
c = torch.from_numpy(Y)
print(b.size())
X_train, X_test, y_train, y_test = train_test_split(b, c, test_size=0.1, random_state=42)
print(y_train[0])


torch.Size([1000, 8, 1, 16, 1])
tensor([13])


In [ ]:
# 基于分布式光纤传感器的

In [ ]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,8,3,padding=2)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8,16,3,padding=2)
        self.avgpool = nn.AvgPool2d(kernel_size=2,stride=2)
        self.fla = nn.Flatten()
        self.fc1 = nn.Linear(640,1024)
        self.fc2 = nn.Linear(1024,20)
        # self.conv1 = nn.Conv2d(1,6,5,padding=2)
        # self.pool = nn.MaxPool2d(2,2)
        # self.conv2 = nn.Conv2d(6,16,3,padding=2)
        # self.avgpool = nn.AvgPool2d(kernel_size=2,stride=2)
        # self.fla = nn.Flatten()
        # self.fc1 = nn.Linear(5*48,1024)
        # self.fc2 = nn.Linear(1024,20)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = self.pool(x)
        # tanh 较好
        x = self.conv2(x)
        x = F.tanh(x)
        x = self.pool(x)

        # x = x.view(-1, (28*28)//(4*4)*8)
        x = x.view(-1,640)
        x = self.fc1(x)
        x = F.tanh(x)

        x = F.dropout(x, p=0.3,training=self.training)
        x = self.fc2(x)
        return x

    def feature_maps(self, x):
        map1 = self.conv1(x)
        map1 = F.tanh(map1)
        map2 = self.pool(map1)
        map2 = self.conv2(map2)
        # map2 = F.relu(map2)
        map2 = F.tanh(map2)
        return (map1, map2)


In [119]:
predicted = torch.tensor([[1,2,3,4]]).float()
target = torch.tensor([1]).long()
lossfxn = nn.CrossEntropyLoss()
loss = lossfxn(predicted, target)
print(loss) # outputs tensor(2.4402)
print(predicted)
print(target)

tensor(2.4402)
tensor([[1., 2., 3., 4.]])
tensor([1])


In [114]:
net = model()
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(net.parameters(),lr = 0.001,momentum=0.9)
print(b.shape)
for epoch in range(20):
    for i,da in enumerate(X_train):
        x = da
        # print(x)
        y = y_train[i]
        # y = torch.tensor([1]).long()
        # y = y.unsqueeze(-2)
        # print(y)
        net.train()#打开dropout
        pred=net(x)
        # print(pred)
        loss=loss_fn(pred,y)
        # print(i,loss)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(epoch,loss)

torch.Size([1000, 8, 1, 16, 1])
0 tensor(2.7175, grad_fn=<NllLossBackward>)
1 tensor(2.4992, grad_fn=<NllLossBackward>)
2 tensor(2.2202, grad_fn=<NllLossBackward>)
3 tensor(1.9326, grad_fn=<NllLossBackward>)
4 tensor(1.2513, grad_fn=<NllLossBackward>)
5 tensor(0.8225, grad_fn=<NllLossBackward>)
6 tensor(0.5927, grad_fn=<NllLossBackward>)
7 tensor(0.4006, grad_fn=<NllLossBackward>)
8 tensor(0.2770, grad_fn=<NllLossBackward>)
9 tensor(0.5465, grad_fn=<NllLossBackward>)
10 tensor(0.2614, grad_fn=<NllLossBackward>)
11 tensor(0.3155, grad_fn=<NllLossBackward>)
12 tensor(0.4322, grad_fn=<NllLossBackward>)
13 tensor(0.3573, grad_fn=<NllLossBackward>)
14 tensor(0.1938, grad_fn=<NllLossBackward>)
15 tensor(0.2373, grad_fn=<NllLossBackward>)
16 tensor(0.1336, grad_fn=<NllLossBackward>)
17 tensor(0.0477, grad_fn=<NllLossBackward>)
18 tensor(0.1081, grad_fn=<NllLossBackward>)
19 tensor(0.0505, grad_fn=<NllLossBackward>)


In [115]:
def accuracy(a,b):
    acc  = 0
    for i,da in enumerate(a):
        x = da
        # print(x)
        y = b[i]
        pred=net(x)
        pred = pred.squeeze()
        # print(pred)
        m = max(pred)
        pre_list = pred.tolist()
        ind = pre_list.index(m)
        y_list = y.tolist()
        y_max = max(y_list)
        y_ind = y_list.index(y_max)

        if ind == y_ind:
            acc += 1
        # print(ind, y_ind)
    print(acc/len(a))

In [118]:
accuracy(X_test,y_test)
accuracy(X_train,y_train)


0.09
0.06333333333333334
